# WINDNODE ABW - Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__authors__ 		= "[Guido Pleßmann](https://github.com/gplssm), [Jonathan Amme](https://github.com/nesnoj), [Julian Endres](https://github.com/nailend), " <br>

## Intro

<font size="4">This jupyter notebook provides plots and information to the results of the dispatch-optimization of certain [scenarios](#0_scenario_information) by the study [__"A regional energy system model for Anhalt-Bitterfeld-Wittenberg"__](https://windnode-abw.readthedocs.io/en/dev/). The different scenarios cover various combinations of renewable energy penetration and flexibility options. The notebooks will, therefore, give an overall view of energy supply and demand by the various scenarios and an insight into scenario-specific distribution and flexiblity effects. This representation is intended to ensure transparency but also to provide a low barrier for further analysis. </font>



<div class="alert alert-warning">
<b>Notes on plots</b>

- Some plots are generated with plotly and may not show up initially as Javascript is not enabled by default.
- This can be solved by _clicking File_ -> __"Trust Notebook"__.
  
These plots have interactive features:
    
- hoovering over the plot will display additional infos 
- clicking the legend selects data
</div>

## Table of Contents

* [0 Scenario information](#0_scenario_information)
* [1 Demand and Generation](#1_demand_and_generation)
* [2 Area required by RES](#2_area_required_by_res)
* [3 Electricity Autarky and Exchange](#3_electrical_autarky_exchange)
* [4 Energy Mix](#4_energy_mix)
* [5 Emissions](#5_emissions)
* [6 Costs](#6_costs)
* [7 Power Grids](#7_power_grids)
* [8 Flexibility](#8_flexibility)
* [9 Energy Exchange](#9_energy_exchange)

In [ ]:
######## WINDNODE ###########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()
# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')
# import scripts
from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *

######## DATA ###########
import re
import pandas as pd

######## PLOTTING ###########
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
# set seaborn style
sns.set()
# plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# 0 Scenario information<a class="anchor" id="0_scenario_information"></a>

In [ ]:
# obtain processed results
regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                                      scenarios=[scenario],
                                      force_new_results=force_new_results)

### set naming and color parameters

In [ ]:
# Names of Municialities
MUN_NAMES = regions_scns[scenario].muns.gen.to_dict()
# extend for total ABW region
MUN_NAMES.update({100:'ABW'})
# Colormap
CMAP = px.colors.sequential.GnBu_r
#UNITS = {"relative": "%", "hours": "h", "Utilization Rate":"%", "Total Cycles":"times", "Full Load Hours":"h"}

# 1 Demand and Generation (Input Data)<a class="anchor" id="1_demand_and_generation"></a>

## 1.1 Installed Electrical Capacities, Municipalities
The following figure shows the installed electrical capacities per technology for each Municipality.

In [ ]:
df_data = results_scns[scenario]['parameters']['Installed capacity electricity supply']
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(3,3, figsize=(12,10))
for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='MW')
    
fig.suptitle('Installed el. Generation Capacity',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 1.2 Electrical Demand
The following figure shows the electrical demand per sectors for each Municipality.

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Stromnachfrage'].sum(level=1)
df_data = df_data.drop(columns='export')
df_data = df_data.rename(columns=PRINT_NAMES)
df_data = df_data / 1e3

fig, axes = plt.subplots(1,3, figsize=(14,4))
for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='GWh')

fig.suptitle('Electrical Demand per Sector',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 1.3 Thermal Demand
The following figure shows the thermal demand per sectors for each Municipality.

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Wärmenachfrage'].sum(level=2)
df_data = df_data.rename(columns=PRINT_NAMES)
df_data = pd.DataFrame([df_data.iloc[:,:2].sum(axis=1).rename('Households'), df_data.iloc[:,-1]]).T
df_data = df_data / 1e3

fig, axes = plt.subplots(1,2, figsize=(14,4))
for ax, (key, data) in  zip(axes.flat, df_data.items()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='GWh')
    
fig.suptitle('Thermal Demand per Sector',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 1.4 installed Capacities, Electricity/Heat, ABW Region

In [ ]:
cap_heat = results_scns[scenario]['parameters']['Installed capacity heat supply'].sum(axis=0).rename('heat')
cap_electricity = results_scns[scenario]['parameters']['Installed capacity electricity supply'].sum(axis=0).rename('electricity')
cap = pd.concat([cap_electricity], keys=['Electricity']).append(pd.concat([cap_heat], keys=['Heat']))

cap = cap.rename(index=PRINT_NAMES)
cap = cap.sort_values(ascending=True)

fig = go.Figure()
for color, (key, df) in enumerate(cap.groupby(level=0)):
    
    visible = True if key == 'Electricity' else 'legendonly'
    fig.add_trace(go.Bar(x=df[key],
                         y=df[key].index,
                         name=key,
                        orientation='h',
                         marker_color=CMAP[2*color],
                         visible=visible,
                         showlegend=True))

    
fig.update_layout(title='Installed Capacities, ABW Region',
                  barmode='stack', legend={'traceorder':'normal'},
                  uniformtext_mode='hide', hovermode="y unified"
                 )
fig.update_traces(hovertemplate='Type: %{fullData.name} <br>' +
                  'Capacity: %{x:.1f} MW <br>'+
                  '<extra></extra>',) #
fig.update_xaxes(title_text='MW')
fig.update_yaxes(title_text='')
fig.show()

## 1.5 Generated Heat/Electricity

In [ ]:
gen_heat = results_scns[scenario]['flows_txaxt']['Wärmeerzeugung'].sum(axis=0).rename('heat')
gen_electricity = results_scns[scenario]['flows_txaxt']['Stromerzeugung'].sum(axis=0).rename('electricity')
gen = pd.concat([gen_electricity], keys=['Electricity']).append(pd.concat([gen_heat], keys=['Heat']))
gen = gen / 1000 # to GWH

gen = gen.rename(index=PRINT_NAMES)
gen = gen.sort_values(ascending=True)

fig = go.Figure()
for color, (key, df) in enumerate(gen.groupby(level=0)):
    
    visible = True if key == 'Electricity' else 'legendonly'
    fig.add_trace(go.Bar(x=df[key],
                         y=df[key].index,
                         name=key,
                        orientation='h',
                         marker_color=CMAP[2*color],
                         visible=visible,
                         showlegend=True))

    
fig.update_layout(title='Generation, ABW Region',
                  barmode='stack', legend={'traceorder':'normal'},
                  uniformtext_mode='hide', hovermode="y unified"
                 )
fig.update_traces(hovertemplate='Type: %{fullData.name} <br>' +
                  'Energy: %{x:.1f} GWh <br>'+
                  '<extra></extra>',) #
fig.update_xaxes(title_text='GWh')
fig.update_yaxes(title_text='')
fig.show()

# 2 Area required by RES<a class="anchor" id="2_area_required_by_res"></a>

## 2.1 Absolute Area

The following figure shows the total absolute area required by the RES in ABW.

Note: For status quo the required area for ground-mounted PV and wind turbines is unknown and therefore not displayed.

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Area required']

# drop pv ground and wind areas for status quo
if regions_scns[scenario].cfg['scn_data']['general']['year'] == 2017:
    df_data.drop(columns=['pv_ground', 'wind'], inplace=True)
    plt_count_y = 1
else:
    plt_count_y = 2

df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(plt_count_y, 2, figsize=(12,6))

for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit='ha')
    
fig.suptitle('Required Area',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 2.2 Relative Area

The following figure shows the total relative area required by the RES in ABW for the current scenario compared to the available areas for different land use scenarios, per RES technology.

Notes:
- The current scenarios of wind turbines and ground-mounted PV is marked with "(current)"
- Technology-specific naming conventions of land use scenarios
  - Wind: Distance to settlements (500m/1000m), use of forests (with/without), percentage of available area due to restrictions resulting from case-by-case decisions
  - PV ground: Restrictions that apply (hard: H / hard+weak: HS), percentage of total available agricultural area
- For status quo the required area for ground-mounted PV and wind turbines is unknown and therefore not displayed.

In [ ]:
df_data = results_scns[scenario]['highlevel_results']

fig = go.Figure()

# PV rooftop
mask = [i for i in df_data.index if 'rel. PV rooftop' in  i[0]]
data = df_data.loc[mask]
index = data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,
           orientation='h',
           name='PV rooftop',
           marker_color=CMAP[5]))

# PV Ground
mask = [i for i in df_data.index if 'rel. PV ground' in  i[0]]
data = df_data.loc[mask]
data.index = data.index.get_level_values(0).str.replace('Area required rel. PV ground \(THIS SCENARIO\)',
                                                        '<b>Area required rel. PV ground (THIS SCENARIO)</b>')
index = data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,
           orientation='h',
           name='PV Ground',
           marker_color=CMAP[4]))#, visible='legendonly'))

# Wind
mask = [i for i in df_data.index if 'rel. Wind' in  i[0]]
data = df_data.loc[mask]
data.index = data.index.get_level_values(0).str.replace('Area required rel. Wind \(THIS SCENARIO\)',
                                                        '<b>Area required rel. Wind (THIS SCENARIO)</b>')
index = data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,
           orientation='h',
           name='Wind',
           marker_color=CMAP[3]))#, visible='legendonly'))

fig.update_layout(title_text = 'Relative Required Area',
                  xaxis=dict(title=' %',
                    titlefont_size=12),
                    autosize=True)

fig.show()

# 3 Electrical Autarky and Exchange<a class="anchor" id="3_electrical_autarky_exchange"></a>

Autarky describes the ratio of the amount of generated eletricity to the demand within the region Anhalt-Bitterfeld-Wittenberg (ABW). A distinction is made depending on the spatial resolution:
  
- **Annual Autarky (Municipality)**: degree of autark electricity supply per municipality disregarding dimension of time
  $$Autarky_{Annual,mun,\%} = \frac{\sum_{t=1}^{8760} E_{supply,ABW,t}}{\sum_{t=1}^{8760} E_{demand,ABW,t}} \cdot 100\,\%$$
  
- **Annual Autarky (ABW)**: degree of autark electricity supply for ABW disregarding dimension of time
  $$Autarky_{Annual,ABW,\%} = \frac{\sum_{t=1}^{8760} E_{supply,mun,t}}{\sum_{t=1}^{8760} E_{demand,mun,t}} \cdot 100\,\%$$

The hours in which the electricity demand in entirely served by local supply is determined by

- **Autarky (ABW)**:
  $$Autark\,hours_{Annual,ABW,\%} = \frac{\sum_{t=1}^{8760} (\frac{E_{supply,ABW,t}}{E_{demand,ABW,t}} \geq 1)}{8760}   \cdot 100\,\%$$

- **Autarky (Municipality)**:
  $$Autark\,hours_{Annual,mun,\%} = \frac{\sum_{t=1}^{8760} (Autarky_{mun,t} \geq 1)}{8760} \cdot 100\,\%$$

  where degree of autarky electricity supply per municipality for each hour is defined as

  $$Autarky_{mun,t} = 1-\frac{E_{import,ext,t}+E_{import,reg,t}+E_{bat,discharge,t}}   {E_{demand,t}+E_{export,ext,t}+E_{export,reg,t}+E_{bat,charge,t}}$$
  
  where:
      
    - $E_{import,ext,t}$, $E_{export,ext,t}$: Imported/exported energy from/to national grid

    - $E_{import,reg,t}$, $E_{export,reg,t}$: Imported/exported energy from/to region's grid (intra-regional)

    - $E_{bat,discharge,t}$: Battery discharge

    - $E_{demand,t}$: Electrical demand

    - $E_{bat,charge,t}$: Battery charge
      

## 3.1 Autarky anually

In [ ]:
fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.17, column_widths=[0.8, 0.2],
                    specs=[[{"secondary_y": True}, {"secondary_y": True}]]
                   )

# For each municipality
fig.add_trace(
    go.Bar(
        x=results_scns[scenario]['results_axlxt']['Autarky'].rename(index=MUN_NAMES).index.tolist(),
        y=results_scns[scenario]['results_axlxt']['Autarky'].values.tolist(),
           orientation='v',
           name='Annual',
           marker_color=CMAP[2],
          ),
    row=1, col=1, secondary_y=False,)

fig.add_trace(
    go.Bar(
        x=results_scns[scenario]['results_axlxt']['Autark hours'].rename(index=MUN_NAMES).index.tolist(),
        y=results_scns[scenario]['results_axlxt']['Autark hours'].values.tolist(),
           orientation='v',
           name='Autark hours',
           marker_color=CMAP[0],
          ),
    row=1, col=1, secondary_y=False,)


# For entire ABW region
fig.add_trace(
    go.Bar(x=["Annually", "Autark supplied hours"], 
           y=[float(results_scns[scenario]['highlevel_results']['Autarky']), 
              float(results_scns[scenario]['highlevel_results']['Autark hours'])],
           orientation='v',
           name='ABW avg.',
           marker_color=CMAP[4],
          ),
    row=1, col=2, secondary_y=False,)

# === Layout ===
fig.update_layout(title_text = 'Electrical Autarky per Municipality (Energy Balance) and total region',
                    autosize=True,
                hovermode="x unified",
                  legend=dict(orientation="h",
                                yanchor="bottom",
                                y=1.02,
                                xanchor="right",
                                x=1),
                 )
fig.update_yaxes(title_text="%", row=1, col=1, anchor="x", secondary_y=False)
fig.update_yaxes(title_text="%", row=1, col=2, anchor="x2", secondary_y=True)

## 3.2 Autarky Geoplots

In [ ]:
df_data = pd.concat([results_scns[scenario]['results_axlxt']['Autarky'].rename('Autarky'), results_scns[scenario]['results_axlxt']['Autark hours'].rename('Autark hours')], axis=1)

fig, axes = plt.subplots(1,2, figsize=(12,5))

for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    
    plot_geoplot(key, data, regions_scns[scenario], ax=ax, unit="%")
    
fig.suptitle('Relative Autarky Balance and Autark Hours',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## 3.3 Degree of autarky distribution
First, we look on the distribution of degree of autarky as average for the entire region.

In [ ]:
data = results_scns[scenario]['flows_txaxt']['Autarky'].mean(level="timestamp")

fig = go.Figure()
fig.add_trace(
    go.Violin(x=data.values, 
              name="ABW average",
              orientation='h',
              line_color=CMAP[1],
              showlegend=False)
)
fig.update_xaxes(title='%')

The degree of autarky may vary significantly among the municipalities.

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Autarky'].unstack()

df_data = df_data.rename(columns=MUN_NAMES)

limit = 2

fig = make_subplots(rows=2, cols=2, horizontal_spacing=0.25,
                    row_heights=[0.8, 0.2],
                    specs=[[{}, {}], [{"colspan": 2}, None]],)

for ags, data in df_data.iteritems():
    
    if data.describe().loc['mean'] > limit:

        fig.add_trace(go.Violin(x=data.values, name=ags,
                                orientation='h',
                               line_color=CMAP[1],
                               showlegend=False),
                              
                      row=1, col=1)
    else:
        fig.add_trace(go.Violin(x=data.values, name=ags,
                                orientation='h',
                               line_color=CMAP[1],
                               showlegend=False),
                      row=1, col=2)
        
    fig.add_trace(go.Violin(x=data.values, name=ags,
                                orientation='h',
                               line_color=CMAP[1],
                               showlegend=True,
                           visible='legendonly'),
                              
                      row=2, col=1)
    
fig.update_xaxes(title_text="%", side='bottom', row=2, col=1)
fig.update_yaxes(type='category', row=1, col=1)
fig.update_yaxes(type='category', row=1, col=2)

fig.update_layout(
    title='Relative Electrical Autarky per Municipality (Energy Balance)',
)
fig.show()

## 3.4 Intra-regional Exchange Balance

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Intra-regional exchange']
df_data = df_data.rename(index=MUN_NAMES)
df_data = df_data / 1e3

fig = go.Figure()

fig.add_trace(go.Bar(x=df_data.index, y=df_data['export'].values, 
                base=0,
                marker_color='crimson',
                name='export'
                ))


fig.add_trace(go.Bar(x=df_data.index, y=df_data['import'].values, 
                base=0,
                marker_color='lightslategrey',
                name='import'
                ))
fig.update_layout(title='Annual net electricity exchanges among administrive districts within ABW region')
fig.update_xaxes(type='category', tickangle=45)
fig.show()

## 3.5 Heatmap

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Stromnetzleitungen']

# swap index of negative values
invert = df_data.loc[df_data['in']<0]
invert.index = invert.index.swaplevel()
invert.index.names = (['ags_from', 'ags_to'])
df_data = df_data.loc[df_data['in']>0].append(invert * -1)

df_data = df_data['out']
df_data = df_data.sort_index()
df_data = df_data / 1e3

hover_text = [f' From: {MUN_NAMES[int(ags_from)]} <br> \
To: {MUN_NAMES[int(ags_to)]} <br> \
Energy: {round(value,2)} MWh' for (ags_from, ags_to), value in df_data.items()]

x = df_data.index.get_level_values(level='ags_from')
x = pd.Series(x.astype(int).values).map(MUN_NAMES).values

y = df_data.index.get_level_values(level='ags_to')
y = pd.Series(y.astype(int).values).map(MUN_NAMES).values

fig = go.Figure(go.Heatmap(
    x=x,
    y=y,
    z=df_data.values,
    colorbar=dict(title='MWh'),
    colorscale='Viridis',
    text= hover_text,
    hoverongaps=False, 
    hovertemplate='%{text}<extra></extra>'
))
fig.update_layout(title='Intra Regional Energy Exchange')
fig.update_xaxes(type='category')
fig.update_yaxes(type='category')
fig.show()

## 3.6 Energy Flows

In [ ]:
df_data = results_scns[scenario]['results_axlxt']['Stromnetzleitungen']
invert = df_data.loc[df_data['in']<0]
invert.index = invert.index.swaplevel()
invert.index.names = (['ags_from', 'ags_to'])
df_data = df_data.loc[df_data['in']>0].append(invert * -1)


converter = dict(zip(MUN_NAMES.keys(), range(20)))

source = [converter[int(i)] for i in df_data['out'].index.get_level_values(level='ags_from')]
target = [converter[int(i)] for i in df_data['out'].index.get_level_values(level='ags_to')]

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad = 70,
        thickness = 10,
        line = dict(
            color = "black", width = 0.5),
        label = [MUN_NAMES[i] for i in list(converter)],
        color = "silver", 
    ),
    link = dict(
        source = source,
        target = target,
        value = df_data['out'].values,
        color = [i for i in COLORS.values()],
        hovertemplate='From: %{source.label}<br />'+
            'to: %{target.label}<br />'+
            'value: %{value:.2f} MWh',#+'<extra></extra>',  
  ))])

fig.update_layout(title_text="Energy Flows",
                  font_size=12,
                  hovermode='x')
fig.show()

# 4 Energy Mix<a class="anchor" id="4_energy_mix"></a>
## 4.1 Region's sum
### 4.1.1 Elecricity

In [ ]:
idx = ['Supply', 'Demand']
el = pd.DataFrame([results_scns[scenario]['results_axlxt']['Stromerzeugung nach Gemeinde'].sum(),
                   pd.concat([results_scns[scenario]['results_axlxt']['Stromnachfrage nach Gemeinde'].sum(), 
                  results_scns[scenario]['results_axlxt']['Stromnachfrage Wärme nach Gemeinde'].sum()])], index=idx) / 1e3

colors = [COLORS[c] for c in el.columns]
el = el.rename(columns=PRINT_NAMES)

fig = px.bar(el, orientation='v',
             title='Electricity supply and demand in ABW region',
             color_discrete_sequence=colors)

fig.update_layout(barmode='stack', legend={'traceorder':'reversed'},
                  uniformtext_mode='hide',
                  autosize=True,
                  legend_title="",
                 )
fig.update_traces(hovertemplate='%{fullData.name}<br>'+
                  '%{y:.1f} GWh <br>'+
                  '<extra></extra>',) # 
fig.update_yaxes(title_text='GWh')
fig.update_xaxes(title_text='')

### 4.1.2 Heat

In [ ]:
idx = ['Supply', 'Demand']
th = pd.DataFrame([results_scns[scenario]['results_axlxt']['Wärmeerzeugung nach Gemeinde'].sum(),
                   results_scns[scenario]['results_axlxt']['Wärmenachfrage nach Gemeinde'].sum()], index=idx) / 1e3

colors = [COLORS[c] for c in th.colhttp://localhost:8888/notebooks/WindNODE_ABW/windnode_abw/jupy/templates/scenario_analysis_template.ipynb#4.2-Balanceumns]
th = th.rename(columns=PRINT_NAMES)

fig = px.bar(th, orientation='v',
             title='Heat supply and demand in ABW region',
             color_discrete_sequence=colors)

fig.update_layout(barmode='stack', legend={'traceorder':'reversed'},
                  uniformtext_mode='hide',
                  autosize=True,
                  legend_title="",
                 )
fig.update_traces(hovertemplate='%{fullData.name}<br>'+
                  '%{y:.1f} GWh <br>'+
                  '<extra></extra>',) # 
fig.update_yaxes(title_text='GWh')
fig.update_xaxes(title_text='')

## 4.2 Balance

In [ ]:
supply = results_scns[scenario]['flows_txaxt']['Stromerzeugung'].sum(level=1)
abw_import = results_scns[scenario]['flows_txaxt']['Intra-regional exchange']['import'].sum(level=1)
abw_import = abw_import.rename('ABW-import')
supply = supply.join(abw_import)


demand = results_scns[scenario]['flows_txaxt']['Stromnachfrage'].sum(level=1)
abw_export = results_scns[scenario]['flows_txaxt']['Intra-regional exchange']['export'].sum(level=1)
abw_export = abw_export.rename('ABW-export')
demand = demand.join(abw_export)
el_heating = results_scns[scenario]['flows_txaxt']['Stromnachfrage Wärme'].sum(level=2).sum(axis=1)
demand = demand.join(el_heating.rename('el_heating'))
    
plot_snd_total(regions_scns[scenario], supply , demand)

## 4.3 Full load hours

In [ ]:
df = (results_scns[scenario]['flows_txaxt']['Stromerzeugung'].drop(columns='import').sum(level=1).sum(axis=0) /
      results_scns[scenario]['parameters']['Installed capacity electricity supply'].sum(axis=0)).fillna(0)

df = df.rename(index=PRINT_NAMES)
df = df.sort_values(ascending=True)

fig = px.bar(df, orientation='h',
             title='Full Load Hours',
             color_discrete_sequence=CMAP,
             # text=''#df.sum(axis=1).to_list())
            )
fig.update_layout(barmode='stack', showlegend=False, legend={'traceorder':'reversed'},
                  uniformtext_mode='hide'#, hovermode="y unified"
                 )
fig.update_traces(hovertemplate='%{fullData.name}<br>'+
                  #'Type: %{y}<br>'+
                  'FLH: %{x:.1f}h <br>'+
                  #'Total: %{text:.1f}€'+
                  '<extra></extra>',) # 
fig.update_xaxes(title_text='h')
fig.update_yaxes(title_text='')
fig.show()

## 4.4 Timeseries

In [ ]:
plot_timeseries(results_scns[scenario], kind='el')#, ags='15091160')

# 5 Emissions<a class="anchor" id="5_emissions"></a>

TODO
- [ ] Notiz zur Zuordnung von KWK zu el etc

## 5.1 Overview

In [ ]:
data_el = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['CO2 emissions el. fix'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['CO2 emissions el. var'].sum(axis=0),
}).rename(index=PRINT_NAMES).reset_index().rename(columns={'index': 'technology'})
data_el['sector'] = 'Power'

data_th = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['CO2 emissions th. fix'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['CO2 emissions th. var'].sum(axis=0),
}).rename(index=PRINT_NAMES).reset_index().rename(columns={'index': 'technology'})
data_th['sector'] = 'Heat'

data_stor_el = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['CO2 emissions stor el. fix'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['CO2 emissions stor el. var'].sum(axis=0),
}).rename(index=PRINT_NAMES)
data_stor_el.index.name = None
data_stor_el = data_stor_el.reset_index().rename(columns={'index': 'technology'})
data_stor_el['sector'] = 'Power'

data_stor_th = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['CO2 emissions stor th. fix'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['CO2 emissions stor th. var'].sum(axis=0),
}).rename(index=PRINT_NAMES)
data_stor_th.index.name = None
data_stor_th = data_stor_th.reset_index().rename(columns={'index': 'technology'})
data_stor_th['sector'] = 'Heat'

data_grid = pd.DataFrame(columns=data_el.columns)
data_grid.loc[0,'fix'] = results_scns[scenario]['results_axlxt']['CO2 emissions grid total'].sum(axis=0) + \
                         results_scns[scenario]['results_axlxt']['CO2 emissions grid new total'].sum(axis=0)
data_grid.loc[0,'var'] = 0
data_grid.loc[0,'technology'] = 'Grid'
data_grid.loc[0,'sector'] = 'Grid'

data = pd.concat([data_el, data_th, data_stor_el, data_stor_th, data_grid],
                 axis=0)

data_fix = data.copy()
data_var = data.copy()
data_fix = data_fix.drop(columns=['var']).rename(columns={'fix': 'emissions'})
data_fix['type'] = 'fix'
data_var = data_var.drop(columns=['fix']).rename(columns={'var': 'emissions'})
data_var['type'] = 'var'


data = pd.concat([data_fix, data_var]).reset_index(drop=True)

fig = px.sunburst(data,
                  #path=['sector', 'technology', 'type'],  # use this param to show split between var+fix
                  path=['sector', 'technology'],
                  values='emissions', color_discrete_sequence=CMAP)
fig.show()

## 5.2 Emissions per Sector

In [ ]:
df_data_left = results_scns[scenario]['results_t']['CO2 emissions th. total'].rename('th').to_frame()
df_data_right = results_scns[scenario]['results_t']['CO2 emissions el. total'].rename('el').to_frame()

# drop nans & zeros
df_data_left = df_data_left[df_data_left!=0].dropna()
df_data_right = df_data_right[df_data_right!=0].dropna()

df_data = df_data_left.join(df_data_right, how='outer')
df_data = df_data.fillna(0).T
df_data = df_data.sort_values(by=list(df_data.index), axis=1, ascending=True)
#df_rel = df_data.T / df_data.sum(axis=1).values

fig = px.bar(df_data, orientation='h',
             color_discrete_sequence=CMAP,
             text=df_data.sum(axis=1).to_list()
            )


fig.update_layout(barmode='stack',
                  autosize=True,
                  title='CO2 Emissions',
                  legend={'traceorder':'reversed'},
                 uniformtext_mode='hide')
fig.update_traces(hovertemplate='CO2: %{x:.1f} t <br>Type: %{y}<br>Total: %{text:.1f} t') # 
fig.update_xaxes(title_text='t CO2')
fig.update_yaxes(title_text='')
fig.show()

## 5.3 Emissions per Sector and Type

In [ ]:
# Electricity
data_el = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['CO2 emissions el. fix'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['CO2 emissions el. var'].sum(axis=0),
})
# Storage el
data_stor_el = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['CO2 emissions stor el. fix'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['CO2 emissions stor el. var'].sum(axis=0),
})
data_el = data_el.append(data_stor_el)
#Grid
data_el.loc['Grid','fix'] = results_scns[scenario]['results_axlxt']['CO2 emissions grid total'].sum(axis=0) + \
results_scns[scenario]['results_axlxt']['CO2 emissions grid new total'].sum(axis=0)

# Heat
data_th = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['CO2 emissions th. fix'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['CO2 emissions th. var'].sum(axis=0),
})
# Storage th
data_stor_th = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['CO2 emissions stor th. fix'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['CO2 emissions stor th. var'].sum(axis=0),
})
data_th = data_th.append(data_stor_th)


# concat
df_data = pd.concat([data_el, data_th,], axis=1,
                    keys=['Electricity Supply', 'Heat Supply'])

df_data = df_data.fillna(0)
#df_data = df_data[(df_data!=0).any(axis=1)]
df_data = df_data.rename(index=PRINT_NAMES)


for sector, df in df_data.groupby(level=0, axis=1):
    df = df[(df!=0).any(axis=1)]
    if df.sum().sum() != 0:
        
        fig = go.Figure()
        for i, (cat, data) in enumerate(df[sector].items()):
            fig.add_trace(go.Bar(x=data.index,
                                 y=data,
                                 name=cat,
                                 marker_color=CMAP[3+i],
                                 hovertemplate='%{y:.1f} t CO2',
                                ))

        fig.update_layout(
            title=f'CO2 Emissions of {sector}',
            barmode='stack',
            hovermode="x unified",
            height=600,
            xaxis={'categoryorder':'category ascending'},
            xaxis_tickfont_size=14,
            yaxis=dict(title='t CO2',
                       titlefont_size=16,
                       tickfont_size=14),
                       autosize=True)
        fig.show()    
    else:
        print(f'no emissions in sector {sector}!')

# 6 Costs<a class="anchor" id="6_costs"></a>

## 6.1 LCOE and LCOH

<div class="alert alert-block alert-info">
<b>Notes on LCOE calculation</b>

- Total LCOE calculate as $LCOE=\frac{expenses_{el.total}}{demand_{el.,total}}$, likewise total LCOH calculate as $LCOH=\frac{expenses_{th.,total}}{demand_{th.,total}}$
- Total expenses $expenses_{el.total}$ are annual expenses. Investment costs are discounted to one year using equivalent periodic costs
- The plot below shows fractions of these LCOE that are calculated as $LCOE_{technology}=\frac{expenses_{el.,technology}}{demand_{el.,total}}$ representation the share of each technology at total cost of one MWh
</div>

In [ ]:
values = ['LCOE','LCOH']

df = pd.DataFrame([results_scns[scenario]['results_t'][i] for i in values], index=values)
df = df.rename(columns=PRINT_NAMES)
df = df.sort_values(by=values, axis=1, ascending=True)

fig = px.bar(df, orientation='h',
             title='LCOE and LCOH',
            color_discrete_sequence=CMAP,
                text=df.sum(axis=1).to_list())

fig.update_layout(barmode='stack', legend={'traceorder':'reversed'},
                  uniformtext_mode='hide'#, hovermode="y unified"
                 )
fig.update_traces(hovertemplate='%{fullData.name}<br>'+
                  'Type: %{y}<br>'+
                  'Share:%{x:.1f}€ <br>'+
                  'Total: %{text:.1f}€'+
                  '<extra></extra>',) # 
fig.update_xaxes(title_text='€/MWh')
fig.update_yaxes(title_text='')
fig.show()

## 6.2 Costs per Sector and Type

In [ ]:
# Electricity
data_el = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['Fix costs el.'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['Variable costs el.'].sum(axis=0),
    'certificats': results_scns[scenario]['results_axlxt']['CO2 certificate cost el.'].sum(axis=0)
})

data_el.loc['Grid','fix'] = results_scns[scenario]['results_axlxt']['Total costs lines'].sum(axis=0) + \
results_scns[scenario]['results_axlxt']['Total costs line extensions'].sum(axis=0)
# Heat
data_th = pd.DataFrame({
    'fix': results_scns[scenario]['results_axlxt']['Fix costs th.'].sum(axis=0),
    'var': results_scns[scenario]['results_axlxt']['Variable costs th.'].sum(axis=0),
    'certificats': results_scns[scenario]['results_axlxt']['CO2 certificate cost th.'].sum(axis=0)
})

# concat
df_data = pd.concat([data_el, data_th], axis=1,
                    keys=['Electricity Supply', 'Heat Supply'])

df_data = df_data.fillna(0)
#df_data = df_data[(df_data!=0).any(axis=1)]
df_data = df_data.rename(index=PRINT_NAMES)
df_data = df_data/ 1e6

for sector, df in df_data.groupby(level=0, axis=1):
    df = df[(df!=0).any(axis=1)]
    if df.sum().sum() != 0:
        
        fig = go.Figure()
        for i, (cat, data) in enumerate(df[sector].items()):
            fig.add_trace(go.Bar(x=data.index,
                                 y=data,
                                 name=cat,
                                 marker_color=CMAP[3+i],
                                 hovertemplate='%{y:.1f} M€'
                                ))

        fig.update_layout(
            title=f'Costs of {sector}',
            hovermode="x unified",
            barmode='stack',
            height=600,
            xaxis={'categoryorder':'category ascending'},
            xaxis_tickfont_size=14,
            yaxis=dict(title='million €',
                       titlefont_size=16,
                       tickfont_size=14),
                       autosize=True)
        fig.show()    
    else:
        print(f'no Costs in sector {sector}!')

# 7 Power Grid<a class="anchor" id="7_power_grid"></a>
## 7.1 Maximum Line Loading Heatmap

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading'].max(level=['ags_from','ags_to'])
df_data = df_data.sort_index(ascending=False)
df_data = df_data * 100

x = df_data.index.get_level_values(level='ags_from')
x = [re.split(r'(\d+)', s) for s in x]
x = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in x]

y = df_data.index.get_level_values(level='ags_to')
y = [re.split(r'(\d+)', s) for s in y]
y = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in y]


fig = go.Figure(go.Heatmap(
    x=x,
    y=y,
    z=df_data.values,
    colorscale=CMAP,
    colorbar=dict(title='%'),
    hovertemplate='-> %{y}<br>'+
    'Max Loading: %{z:.1f}%'+
    '<extra></extra>',
    showscale=True,
    hoverongaps=False, 
))
fig.update_layout(title='Max Line Loading in %',
                  hovermode="x unified")
fig.update_xaxes(type='category')
fig.update_yaxes(type='category', showspikes=True)
fig.show()

## 7.1 Maximum Line Loading Barcharts

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading'].max(level=['ags_from','ags_to']) * 100
df_data = df_data.sort_index(ascending=False)
df_data = pd.DataFrame().from_dict({'line loading': df_data,'free capacity': 100-df_data})


ags_from, ags_to = list(zip(*df_data.index))
ags_from = [s.replace(re.findall(r"\d+",s)[0], MUN_NAMES[int(re.findall(r"\d+",s)[0])]) for s in ags_from]
ags_to = [s.replace(re.findall(r"\d+",s)[0], MUN_NAMES[int(re.findall(r"\d+",s)[0])]) for s in ags_to]

df_data.index = pd.MultiIndex.from_tuples(zip(*(ags_from, ags_to)))

df_data.index = [f'{ags_from} -> {ags_to}' for ags_from, ags_to in df_data.index]
#index = [re.split(r'(\d+)', s) for s in df_data.index]
#df_data.index = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in index]

fig = go.Figure() 
for i, (key, df) in enumerate(df_data.items()):

    fig.add_bar(y=df.index,
                x=df.values,
                orientation='h',
                name=key,
                marker_color=CMAP[4*i+2],
                hovertemplate='%{x:.2f}%',
                showlegend=False)

fig.update_layout(barmode="relative",
    title='Maximum Line Loading',
    yaxis_tickfont_size=12,
    xaxis=dict(
        title='Loading in %',
        titlefont_size=16,
        tickfont_size=12,
    ))

fig.update_yaxes(type='category')#,  tickangle=45)
fig.update_xaxes(showspikes=True)
fig.update_layout(hovermode="y unified")
fig.show()


## 7.2 Line Loading Distribution

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading per bus']
df_data = df_data * 100
index = df_data.index.get_level_values(level=0)

fig = go.Figure()
# limit = 2
# fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.25)
# for ags, data in df_data.iteritems():
#     if data.describe().loc['mean'] > limit0

fig.add_trace(go.Box(
    y=df_data.values,
    x=index,
    name='Increase',
    marker_color=CMAP[3],
    boxpoints=False,))

fig.update_layout(
    title='Line Loading',
    yaxis_title='in %',
    boxmode='group',
    hovermode="x unified")

fig.update_xaxes(type='category', tickangle=45)
fig.show()

## 7.3 Mean Line Loading

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['Line loading'].mean(level=['ags_from','ags_to'])
df_data = df_data.sort_index(ascending=False)
df_data = df_data * 100

hover_text = [ f'Mean loading:{int(i)}%<br>' for i in df_data.values]

x = df_data.index.get_level_values(level='ags_from')
x = [re.split(r'(\d+)', s) for s in x]
x = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in x]

y = df_data.index.get_level_values(level='ags_to')
y = [re.split(r'(\d+)', s) for s in y]
y = [f"{start}{MUN_NAMES[int(ags)]}{end}" for start,ags,end in y]


fig = go.Figure(go.Heatmap(
    x=x,
    y=y,
    z=df_data.values,
    colorscale=CMAP,
    colorbar=dict(title='%'),
    hovertemplate='-> %{y}<br>'+
    'Mean Loading: %{z:.1f}%'+
    '<extra></extra>',
    #hovertext=hover_text,
    showscale=True,
    hoverongaps=False, 
))
fig.update_layout(title='Mean Line Loading in %',
                  hovermode="x unified")
fig.update_xaxes(type='category')
fig.update_yaxes(type='category', showspikes=True)
fig.show()

# 8 Flexibility<a class="anchor" id="8_flexibility"></a>

Calculation of storage ratios:

To compare/show the usage of different flexibility options 3 different ratios are used:

1. Full Discharge Hours:
    
    The Ratio of discharged energy $E_{tech, discharge}$ to nominal discharge power $P_{n, discharge}$
    
    $$Full Discharge Hours_{technology}=\frac{E_{tech, discharge}}{P_{n, discharge}}$$
    
2. Total Cycles:
    
    The Ratio of discharged energy $E_{tech, discharge}$ to installed capacity $C_{technology}$
    
    $$Total Cycles_{technology}=\frac{E_{tech, discharge}}{C_{technology}}$$
    
    
3. Utilization Rate:
    
    The Ratio of $Total Cycles_{technology}$ to $Max Cycles_{technology}$
    
    $$Max Cycles_{technology}=\frac{1}{2} \cdot timesteps \cdot C_{rate}$$
    
    with
    
    $$C_{rate}  = \begin{cases}
    \frac{P_{n, discharge}}{C_{technology}} & \frac{P_{n, discharge}}{C_{technology}} \leq 1 \\
    1 & \, \text{otherwise}
    \end{cases}$$
    
    $$Utilization Rate_{technology} = \frac{Total Cycles_{technology}}{Max Cycles_{technology}}$$

## 8.1 Heat Storage

<div class="alert alert-block alert-info">
<b>Notes on Heatstorage Charts</b>
  
- Relative cycles will not be used for comparison. This is due to a high c-rate (6.7) of small, decentralised storages which makes the relative usage using eq. full cycles not very meaningful.
    
- To show the total cycles, only the discharge values are chosen as the difference are the storage losses and almost negligible.

</div>
    

In [ ]:
capacity = results_scns[scenario]['parameters']['Installed capacity heat storage']
capacity = capacity.rename(columns={'stor_th_large':'cen','stor_th_small':'dec'})

power_discharge = results_scns[scenario]['parameters']['Discharge power heat storage']
power_discharge = power_discharge.rename(columns={'stor_th_large':'cen','stor_th_small':'dec'})

discharge = results_scns[scenario]['results_axlxt']['Wärmespeicher nach Gemeinde']
discharge = discharge.discharge.unstack().fillna(0).T

# combine
heat_storage_figures = pd.concat([capacity, power_discharge, discharge], axis=1,
                                 keys=['capacity', 'power_discharge', 'discharge'])

df_data = heat_storage_figures.sum().unstack()
df_data = df_data.join(df_data.sum(axis=1).rename('Total'))
display(df_data.round())

In [ ]:
heat_storage_ratios = get_storage_ratios(heat_storage_figures, regions_scns[scenario])
heat_storage_ratios = heat_storage_ratios.drop(columns=[('dec', 'Utilization Rate')])

plot_storage_ratios(heat_storage_ratios, regions_scns[scenario], title='Heat storage')

### TODO

- Ratios für Gesamtregion? (als scalare Tabelle) Wie genau soll das gehen?
    - Unterschiedliche Kapazitätswerte. Addition von Cen und Dec nicht sinnvoll.
    - Mittelwerte auch nicht wirklich sinnvoll


## 8.2 Battery Storage

<div class="alert alert-block alert-info">
<b>Notes on Battery Storage Charts</b>
  
- The average total cycles (get visible when activating "ABW" in legend) are calculated as mean of total cycles from each municipality.
    
- To show the total cycles, only the discharge values are chosen. The difference between charge and discharge results from losses and are somewhat negligible.
    
- If no small storages are installed in a scenario, they cannot be evaluated. This will result in empty charts.

</div>
    

### 8.2.1 Utilization

In [ ]:
stor_cap_large = results_scns[scenario]['parameters']['Installierte Kapazität Großbatterien']
stor_cap_small = results_scns[scenario]['parameters']['Installierte Kapazität PV-Batteriespeicher']

battery_storage_figures = pd.concat([stor_cap_large, stor_cap_small], axis=1,
                                    keys=['large','small'])

storage =  results_scns[scenario]['results_axlxt']['Batteriespeicher nach Gemeinde']
storage = storage.unstack("level").swaplevel(axis=1)

battery_storage_figures = battery_storage_figures.join(storage).sort_index(level=0, axis=1)
battery_storage_figures = battery_storage_figures.swaplevel(axis=1)

df_data = battery_storage_figures.sum().unstack()
df_data = df_data.join(df_data.sum(axis=1).rename('Total'))
display(df_data.round())

In [ ]:
battery_storage_ratios = get_storage_ratios(battery_storage_figures, regions_scns[scenario])

plot_storage_ratios(battery_storage_ratios, regions_scns[scenario], title='Battery Storage')

### 8.2.2 Timeseries

In [ ]:
# timeseries
df_data = results_scns[scenario]['flows_txaxt']['Batteriespeicher'].sum(level=0)
# only get nonzero values
#df_data = df_data.loc[(df_data != 0).any(axis=1)]

fig = go.Figure()

for i, (key, df) in enumerate(df_data.items()):
# Add traces
    fig.add_trace(go.Scatter(x=df.index,
                             y=df,
                             mode='markers+lines',
                             line=dict(color=CMAP[2*i+1], width=1,),
                             opacity=0.8,
                             name=key,
                            line_shape='hv',
                            hovertemplate='%{y:.2f} MWh<br>',
                            ))
fig.update_xaxes(
        title='Zoom',
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=14, label="2w", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=3, label="3d", step="day", stepmode="backward"),
                ])))
fig.update_layout(title='Battery Charge/Discharge',
                  hovermode="x unified")
fig.update_yaxes(title_text="Energy in MWh", showspikes=True)
fig.show()

### 8.2.3 Scatter

<div class="alert alert-block alert-info">
<b>Notes on Battery Storage Scatter Chart</b>

- Datapoints with zero values for both discharge or charge of battery storage are excluded
- The label of the Y-axis diverts as its varying units depending on the traces you select. The units will be displayed in the infobox by hovering over the datapoints
- The mean value of all line loadings is used per timestep
    
</div>

In [ ]:
RE = ['pv_ground', 'pv_roof_large', 'pv_roof_small', 'wind']
# timeseries
df_re_ts = results_scns[scenario]['flows_txaxt']['Stromerzeugung'][RE].sum(axis=1).sum(level=0)
df_dsm_ts = results_scns[scenario]['flows_txaxt']['DSM activation']['Demand decrease'].sum(level=0)
df_imports_ts = results_scns[scenario]['flows_txaxt']['Stromimport'].sum(axis=1).sum(level=0)
df_lines_ts = results_scns[scenario]['flows_txaxt']['Line loading'].mean(level=2)

df_storage_in_ts = results_scns[scenario]['flows_txaxt']['Batteriespeicher']['charge'].sum(level=0)
df_storage_out_ts = results_scns[scenario]['flows_txaxt']['Batteriespeicher']['discharge'].sum(level=0)

df_data = pd.concat([df_re_ts, df_dsm_ts, df_imports_ts, df_lines_ts, df_storage_in_ts, df_storage_out_ts],
                    axis=1, keys=['RE', 'DSM', 'Import', 'Lineload', 'Charge', 'Discharge'])

# remove every timesteps where charge/discharge equals zero
df_data = df_data.loc[(df_data[['Charge', 'Discharge']] != 0).any(axis=1)]

fig = go.Figure()

for i, (key, df) in enumerate(df_data.drop(columns=['Charge', 'Discharge']).items()):
    
    visible = 'legendonly' if i else True
    
    hovertemplate = "%{fullData.name}<br>x = %{x:.2f} MWh <br>y = %{y:.2f} " + UNITS[key]
    
    
    # Add traces
    fig.add_trace(go.Scatter(y=df,
                             x=df_data['Charge'],
                             mode='markers',
                             opacity=0.7,
                             marker_color=CMAP[3],
                             name= 'Charge-'+key,
                             visible=visible,
                             hovertemplate=hovertemplate,
                             ))

    fig.add_trace(go.Scatter(y=df,
                             x=df_data['Discharge'],
                             mode='markers',
                             marker_color=CMAP[0],
                             opacity=0.7,
                             name= 'Disharge-' + key,
                             visible=visible,
                             hovertemplate=hovertemplate,
                            ))



fig.update_layout(title="Scatter Battery Storage - X",)
fig.update_yaxes(title_text='MWh or % for line loading', showspikes=True)
fig.update_xaxes(title_text='MWh of battery storage charge/discharge', showspikes=True)
fig.show()

## 8.3 DSM

### 8.3.1 Activation

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['DSM activation']

x = df_data.index.get_level_values(level=1)
x = pd.Series(x.astype(int).values).map(MUN_NAMES).values

fig = go.Figure()

for key, df in df_data.items():
    
    color = '#3D9970' if 'increase' in key else '#FF4136'
    fig.add_trace(go.Box(x=x,
        y=df,
        name=key,
        marker_color=color,
        boxpoints=False,))

fig.update_layout(
    yaxis_title='DSM activation in MWh',
    boxmode='group',
#    hovermode="x unified",
)

fig.update_xaxes(type='category', tickangle=45)
fig.show()


### 8.3.2 DSM Total Sum

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['DSM activation']
new = pd.concat([df_data.sum(level='timestamp')],axis=1, keys=[100], names=['ags']).swaplevel(axis=1)
df_data = df_data.unstack().join(new)
df_data = df_data.sort_index(axis=1).stack()
df_data = df_data.sum(level='ags')

demand_hh = results_scns[scenario]['results_axlxt']['Stromnachfrage nach Gemeinde']['hh']
demand_hh = demand_hh.append(pd.Series(demand_hh.sum(), index=[100]))

df_data = df_data['Demand decrease'] /  demand_hh * 100

#df_data = df_data.sort_values(ascending=False)
df_data = df_data.rename(index=MUN_NAMES)

fig = go.Figure()
fig.add_trace(
    go.Bar(x=df_data.index,
           y=df_data.values,
           name='Ratio',
           orientation='v',
           marker_color=CMAP[1],
          hovertemplate='%{y:.2f} %'))

fig.update_yaxes(title='%')
fig.update_layout(
    title='Ratio DSM / Electrical Demand of Households',
    showlegend=False,
    hovermode="x unified")
fig.show()

### 8.3.3 DSM Timeseries

In [ ]:
df_data = results_scns[scenario]['flows_txaxt']['DSM activation']
new = pd.concat([df_data.sum(level='timestamp')],axis=1, keys=['100'], names=['ags']).swaplevel(axis=1)
df_data = df_data.unstack().join(new)
df_data = df_data.sort_index(axis=1).stack()

fig = go.Figure()
for vis, (ags, df) in enumerate(df_data.groupby(level='ags')):
    for leg, (key, data) in enumerate(df_data.items()):
        
        legend = False if leg else True
        visible = 'legendonly' if vis else True
        
        data = data.loc[(slice(None), ags)]
        fig.add_trace(go.Scatter(x=data.index,
                                 y=data.values,
                                 name=MUN_NAMES[int(ags)],
                                 legendgroup=ags,
                                 mode='lines',
                                 showlegend=legend,
                                 visible=visible,
                                 marker_color=CMAP[2*leg+2],
                                 text=data.name,
                                 hovertemplate='%{fullData.text}<br>%{y:.2f} MW'
                                ))

fig.update_xaxes(
    title='Zoom',
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=14, label="2w", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=3, label="3d", step="day", stepmode="backward"),
            #dict(step="all")
        ])
    )
)

fig.update_layout(
    title='Demand Side Management of ABW',
    height = 700,
    xaxis_tickfont_size=14,
    yaxis=dict(title='MW', titlefont_size=16, tickfont_size=14),
    autosize=True,
    hovermode="x unified")

fig.show()

### 8.3.4 DSM Relative Utilization

The relative utilization of DSM describes how much positive or negative power activation takes place in relation to the maximum potential assumed in the scenario.

In [ ]:
df_dsm_cap_up, df_dsm_cap_down = calc_dsm_cap(region=regions_scns[scenario])
df_dsm_cap = pd.concat([df_dsm_cap_up.sum().rename('Demand increase'),
                     df_dsm_cap_down.sum().rename('Demand decrease')], axis=1)

df_data = results_scns[scenario]['flows_txaxt']['DSM activation'].sum(level='ags') / df_dsm_cap.values
df_data.index = df_data.index.astype(int)
df_data = df_data.rename(index=MUN_NAMES)
df_data = df_data * 100

fig = go.Figure()
for i, (key, data) in enumerate(df_data.items()):

    fig.add_trace(
        go.Bar(x=data.index,
               y=data.values,
               name=key,
               orientation='v',
               marker_color=CMAP[3+2*i],
              hovertemplate='%{y:.2f} %'))

fig.update_yaxes(title='%')
fig.update_xaxes(type='category')
fig.update_layout(
    title='DSM Relative Utilization',
    hovermode="x unified")
fig.show()